In [27]:
!pip install chardet


In [28]:
import pdfplumber
import pandas as pd
import numpy as np
import re
import chardet
from pathlib import Path
from datetime import datetime
import logging
from typing import Optional, List, Dict
from IPython.display import display

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("CX_Startlist_Parser")


In [29]:
CX_SCHEMA = {
    "event_id": "str",
    "event_date": "str",
    "circuit": "str",              # UCI / SP / X20 / Exact / Other
    "category": "str",             # ME, WE, MU23, WU23, MJ, WJ
    "bib_number": "int64",
    "rider_full_name": "str",
    "rider_first_name": "str",
    "rider_last_name": "str",
    "nationality": "str",
    "team_name": "str",
    "team_code": "str",
    "start_position": "float64",
    "uci_id": "str",
    "year_of_birth": "int64",
    "wcs_points": "float64",
    "uci_rank": "float64",
    "rider_id": "str",
    "source_file": "str",
    "parse_timestamp": "str",
    "data_quality_score": "float64"
}


In [30]:
CATEGORY_MAP = {
    "elite men": "ME",
    "men elite": "ME",
    "men": "ME",
    "me": "ME",
    "elite women": "WE",
    "women elite": "WE",
    "women": "WE",
    "we": "WE",
    "u23 men": "MU23",
    "men u23": "MU23",
    "mu23": "MU23",
    "u23 women": "WU23",
    "women u23": "WU23",
    "wu23": "WU23",
    "junior men": "MJ",
    "mj": "MJ",
    "junior women": "WJ",
    "wj": "WJ"
}

def normalize_category(raw_cat: str) -> str:
    if not raw_cat:
        return ""
    raw = raw_cat.lower().replace("_", " ").strip()
    for k, v in CATEGORY_MAP.items():
        if k in raw:
            return v
    return ""


In [31]:
def detect_circuit(file_path: Path, df_preview: Optional[pd.DataFrame] = None) -> str:
    name = file_path.name.lower()

    # File name heuristics
    if "uci" in name or "worldcup" in name:
        return "UCI"
    if "superprestige" in name or "sp_" in name:
        return "Superprestige"
    if "x20" in name or "telenet" in name:
        return "X20"
    if "exact" in name or "ethias" in name:
        return "Exact Cross"

    # Preview heuristics
    if df_preview is not None:
        cols = " ".join([c.lower() for c in df_preview.columns])
        if "uci id" in cols or "uci_code" in cols:
            return "UCI"
        if "superprestige" in cols:
            return "Superprestige"
        if "telenet" in cols:
            return "X20"

    return "Other"


In [32]:
def detect_encoding(path: Path) -> str:
    try:
        raw = path.read_bytes()
        result = chardet.detect(raw)
        if result["confidence"] > 0.7:
            return result["encoding"]
    except:
        pass
    return "utf-8"

def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Force *everything* to string to avoid .lower() on ints
    df.columns = [str(c) if c is not None else "" for c in df.columns]
    df.columns = [
        re.sub(r"[^\w\s]", "", c.lower()).strip().replace(" ", "_")
        for c in df.columns
    ]
    return df



In [33]:
def parse_uci_pdf(path: Path) -> pd.DataFrame:
    all_tables = []

    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables({
                "vertical_strategy": "lines",
                "horizontal_strategy": "lines",
                "intersection_y_tolerance": 5,
                "intersection_x_tolerance": 5
            })

            if not tables:
                continue

            for t in tables:
                if not t or len(t) < 2:
                    continue

                header = t[0]
                rows = t[1:]

                df = pd.DataFrame(rows, columns=header)

                # Remove duplicate columns before normalization
                df = df.loc[:, ~df.columns.duplicated()]

                all_tables.append(df)

    if all_tables:
        return pd.concat(all_tables, ignore_index=True)
    return pd.DataFrame()

def parse_uci_csv(path: Path) -> pd.DataFrame:
    enc = detect_encoding(path)
    for delim in [",", ";", "\t"]:
        try:
            df = pd.read_csv(path, encoding=enc, delimiter=delim)
            if len(df.columns) > 2:
                return df
        except:
            pass
    return pd.DataFrame()

def parse_uci_excel(path: Path) -> pd.DataFrame:
    try:
        xl = pd.ExcelFile(path)
        for sheet in xl.sheet_names:
            df = pd.read_excel(xl, sheet)
            if len(df.columns) > 2:
                return df
    except:
        pass
    return pd.DataFrame()

def parse_uci_html(path: Path) -> pd.DataFrame:
    try:
        tables = pd.read_html(str(path))
        for t in tables:
            if len(t.columns) > 2:
                return t
    except:
        pass
    return pd.DataFrame()


In [34]:
def parse_superprestige_pdf(path: Path) -> pd.DataFrame:
    """
    Superprestige PDFs almost never contain proper tables.
    They contain lines like:

    12   VANDERPOEL Mathieu   NED   Alpecin-Deceuninck
    """

    rows = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            for line in text.split("\n"):
                if re.match(r"^\s*\d+\s+[A-Za-z]", line):
                    # Split by 2+ spaces
                    parts = re.split(r"\s{2,}", line.strip())
                    rows.append(parts)

    if not rows:
        return pd.DataFrame()

    # Build a rough dataframe
    df = pd.DataFrame(rows)
    df = df.loc[:, ~df.columns.duplicated()]
    return df


In [35]:
def parse_x20_pdf(path: Path) -> pd.DataFrame:
    tables = []

    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table is None:
                continue

            header = table[0]
            data_rows = table[1:]

            df = pd.DataFrame(data_rows, columns=header)
            df = df.loc[:, ~df.columns.duplicated()]
            tables.append(df)

    return pd.concat(tables, ignore_index=True) if tables else pd.DataFrame()


In [36]:
def parse_generic(path: Path) -> pd.DataFrame:
    # Try CSV → HTML → Excel → PDF
    df = parse_uci_csv(path)
    if not df.empty: return df

    df = parse_uci_html(path)
    if not df.empty: return df

    df = parse_uci_excel(path)
    if not df.empty: return df

    df = parse_uci_pdf(path)
    return df


In [37]:
def load_raw_startlist(path: Path, circuit: str) -> pd.DataFrame:
    suffix = path.suffix.lower()

    if circuit == "UCI":
        if suffix == ".pdf": return parse_uci_pdf(path)
        if suffix == ".csv": return parse_uci_csv(path)
        if suffix in [".xlsx", ".xls"]: return parse_uci_excel(path)
        if suffix in [".html", ".htm"]: return parse_uci_html(path)

    if circuit == "Superprestige":
        if suffix == ".pdf": return parse_superprestige_pdf(path)

    if circuit == "X20":
        if suffix == ".pdf": return parse_x20_pdf(path)

    # Exact + others
    return parse_generic(path)


In [38]:
def normalize_cx_schema(df: pd.DataFrame) -> pd.DataFrame:
    df = clean_column_names(df)

    # normalize bib
    df["bib_number"] = (
        df.iloc[:, 0].astype(str).str.extract(r"(\d+)", expand=False)
    )

    # normalize names
    name_col = None
    for c in df.columns:
        if "name" in c or "rider" in c:
            name_col = c
            break

    df["rider_full_name"] = df[name_col].astype(str)

    # split
    parts = df["rider_full_name"].str.split(" ", n=1, expand=True)
    df["rider_first_name"] = parts[0]
    df["rider_last_name"] = parts[1] if parts.shape[1] > 1 else ""

    # nationality
    nat_col = None
    for c in df.columns:
        if "nat" in c or "country" in c:
            nat_col = c
            break
    df["nationality"] = df[nat_col].astype(str).str.upper() if nat_col else ""

    # team
    team_col = None
    for c in df.columns:
        if "team" in c or "club" in c:
            team_col = c
            break
    df["team_name"] = df[team_col] if team_col else ""

    # other fields
    df["uci_id"] = ""
    df["team_code"] = ""
    df["year_of_birth"] = pd.NA
    df["start_position"] = pd.NA
    df["wcs_points"] = pd.NA
    df["uci_rank"] = pd.NA

    return df


In [39]:
def enrich_and_score(
    df: pd.DataFrame,
    file_path: Path,
    circuit: str,
    category: str,
    metadata: Dict
) -> pd.DataFrame:

    df["source_file"] = file_path.name
    df["circuit"] = circuit
    df["category"] = normalize_category(category)

    # metadata overrides filename
    df["event_id"] = metadata.get("event_id", file_path.stem.lower())
    df["event_date"] = metadata.get("event_date", "")

    df["parse_timestamp"] = datetime.now().isoformat()

    # rider ID logic
    df["rider_id"] = df.apply(
        lambda r: r["uci_id"] if r["uci_id"] else f"{r['rider_last_name'][:3].upper()}_{r['nationality']}",
        axis=1
    )

    # quality score
    scores = []
    for _, r in df.iterrows():
        score = 1.0
        if pd.isna(r["bib_number"]): score -= 0.2
        if not r["rider_full_name"]: score -= 0.3
        if not r["nationality"]: score -= 0.1
        if not r["team_name"]: score -= 0.1
        scores.append(max(0, score))

    df["data_quality_score"] = scores

    return df


In [40]:
def detect_category(path: Path, user_meta: Dict) -> str:
    # Metadata overrides
    if "category" in user_meta and user_meta["category"]:
        return user_meta["category"]

    name = path.stem.lower()

    for k in CATEGORY_MAP.keys():
        if k.replace(" ", "_") in name:
            return CATEGORY_MAP[k]

    return ""


In [41]:
def parse_startlist(file_path: str, metadata: Dict = None) -> pd.DataFrame:
    metadata = metadata or {}
    path = Path(file_path)

    # Preview CSV for circuit detection
    try:
        preview = pd.read_csv(path, nrows=5)
    except:
        preview = None

    circuit = metadata.get("circuit") or detect_circuit(path, preview)
    category = detect_category(path, metadata)

    logger.info(f"Detected circuit: {circuit}")
    logger.info(f"Detected category: {category}")

    raw_df = load_raw_startlist(path, circuit)

    if raw_df.empty:
        logger.warning("No rows extracted from startlist.")
        return pd.DataFrame()

    normalized = normalize_cx_schema(raw_df)
    enriched = enrich_and_score(normalized, path, circuit, category, metadata)

    return enriched


In [42]:
def parse_multiple_startlists(paths: List[str], metadatas: List[Dict] = None):
    dfs = []
    for i, p in enumerate(paths):
        meta = metadatas[i] if metadatas and i < len(metadatas) else {}
        df = parse_startlist(p, meta)
        if not df.empty:
            dfs.append(df)
    if dfs:
        return pd.concat(dfs, ignore_index=True)
    return pd.DataFrame()


In [43]:
# ============================================================
# TEST HARNESS FOR ALL STARTLISTS — PRODUCTION VERSION
# ============================================================

import os
from pathlib import Path

startlist_dir = Path("data/raw/startlists")

test_files = [
    startlist_dir / "Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__MEN-ELITE.pdf",
    startlist_dir / "Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__WOMEN-ELITE.pdf",
    startlist_dir / "UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Elite.pdf",
    startlist_dir / "UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Junior.pdf",
    startlist_dir / "UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-U23.pdf",
    startlist_dir / "UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Elite.pdf",
    startlist_dir / "UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Junior.pdf",
    startlist_dir / "X2O-BADKAMERS-TROFEE__RECTAVIT-FLANDRIENCROSS-HAMME__Hamme-BEL__2025-11-16.pdf"
]

# Matching metadata (same order)
metadata_list = [
    {"event_id": "sp_merksplas_2025", "event_date": "2025-11-15"},
    {"event_id": "sp_merksplas_2025", "event_date": "2025-11-15"},
    {"event_id": "uci_wc_tabor_2025", "event_date": "2025-11-23"},
    {"event_id": "uci_wc_tabor_2025", "event_date": "2025-11-23"},
    {"event_id": "uci_wc_tabor_2025", "event_date": "2025-11-23"},
    {"event_id": "uci_wc_tabor_2025", "event_date": "2025-11-23"},
    {"event_id": "uci_wc_tabor_2025", "event_date": "2025-11-23"},
    {"event_id": "x20_hamme_2025",    "event_date": "2025-11-16"}
]

all_results = []

for fpath, meta in zip(test_files, metadata_list):
    print("\n====================================")
    print(f"🔍 Parsing: {fpath.name}")
    print("====================================")

    if not fpath.exists():
        print(f"❌ File NOT FOUND: {fpath}")
        continue

    try:
        df = parse_startlist(str(fpath), meta)
        
        if df.empty:
            print("⚠️ Parsed but returned EMPTY dataset.")
        else:
            print(f"✔️ Parsed {len(df)} riders")
            print(f"📈 Avg Quality: {df['data_quality_score'].mean():.3f}")
            print(f"🏁 Circuit detected: {df['circuit'].iloc[0]}")
            print(f"🎯 Category detected: {df['category'].iloc[0]}")

            display(df.head())

            all_results.append(df)

    except Exception as e:
        print(f"❌ ERROR parsing {fpath.name}: {e}")

# Combine all
if all_results:
    master_df = pd.concat(all_results, ignore_index=True)

    print("\n====================================")
    print("🎉 ALL STARTLISTS COMBINED")
    print("====================================")
    print(f"Total Riders: {len(master_df)}")
    print("Circuits:", master_df['circuit'].unique())
    print("Categories:", master_df['category'].unique())

    master_df.to_csv("all_startlists_combined.csv", index=False)
    print("\n💾 Saved: all_startlists_combined.csv")

    display(master_df.head())
else:
    print("❌ No startlists parsed successfully.")


2025-11-20 09:12:33,425 - INFO - Detected circuit: Superprestige
2025-11-20 09:12:33,426 - INFO - Detected category: ME
2025-11-20 09:12:33,478 - INFO - Detected circuit: Superprestige
2025-11-20 09:12:33,478 - INFO - Detected category: ME
2025-11-20 09:12:33,516 - INFO - Detected circuit: UCI
2025-11-20 09:12:33,516 - INFO - Detected category: ME
2025-11-20 09:12:33,586 - INFO - Detected circuit: UCI
2025-11-20 09:12:33,586 - INFO - Detected category: ME



🔍 Parsing: Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__MEN-ELITE.pdf
❌ ERROR parsing Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__MEN-ELITE.pdf: None

🔍 Parsing: Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__WOMEN-ELITE.pdf
❌ ERROR parsing Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__WOMEN-ELITE.pdf: None

🔍 Parsing: UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Elite.pdf
❌ ERROR parsing UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Elite.pdf: 'NoneType' object is not subscriptable

🔍 Parsing: UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Junior.pdf


2025-11-20 09:12:33,666 - INFO - Detected circuit: UCI
2025-11-20 09:12:33,666 - INFO - Detected category: ME
2025-11-20 09:12:33,723 - INFO - Detected circuit: UCI
2025-11-20 09:12:33,723 - INFO - Detected category: ME
2025-11-20 09:12:33,837 - INFO - Detected circuit: UCI
2025-11-20 09:12:33,837 - INFO - Detected category: ME


❌ ERROR parsing UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Junior.pdf: 'NoneType' object is not subscriptable

🔍 Parsing: UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-U23.pdf
❌ ERROR parsing UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-U23.pdf: 'NoneType' object is not subscriptable

🔍 Parsing: UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Elite.pdf
❌ ERROR parsing UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Elite.pdf: 'NoneType' object is not subscriptable

🔍 Parsing: UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Junior.pdf


2025-11-20 09:12:33,887 - INFO - Detected circuit: Other
2025-11-20 09:12:33,887 - INFO - Detected category: ME


❌ ERROR parsing UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Junior.pdf: 'NoneType' object is not subscriptable

🔍 Parsing: X2O-BADKAMERS-TROFEE__RECTAVIT-FLANDRIENCROSS-HAMME__Hamme-BEL__2025-11-16.pdf
❌ ERROR parsing X2O-BADKAMERS-TROFEE__RECTAVIT-FLANDRIENCROSS-HAMME__Hamme-BEL__2025-11-16.pdf: None
❌ No startlists parsed successfully.


In [24]:
import os

folder = "data/raw/startlists"

print(f"FILES INSIDE {folder}:\n")
for f in os.listdir(folder):
    print(f)


FILES INSIDE data/raw/startlists:

UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-U23.pdf
X2O-BADKAMERS-TROFEE__RECTAVIT-FLANDRIENCROSS-HAMME__Hamme-BEL__2025-11-16.pdf
Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__MEN-ELITE.pdf
UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Junior.pdf
Telenet-Superprestige__AARDBEIENCROSS-MERKSPLAS__Merksplas-BEL__2025-11-15__WOMEN-ELITE.pdf
UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Junior.pdf
UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Men-Elite.pdf
UCI-World-Cup__Tabor__Tabor-Czechia__2025-11-23__Women-Elite.pdf
